In [1]:
# copyright ############################### #
# This file is part of the Xtrack Package.  #
# Copyright (c) CERN, 2021.                 #
# ######################################### #
import xtrack as xt
import xobjects as xo
import xpart as xp

import ducktrack as dtk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from tqdm import tqdm


beta_rel = 0.037
#gamma=  1.00566349259
#mass0=193729.0248722061*1e6 #eV/c2
mass0=938.27208816*1e6 #ev/c^2
c=299792458.0
gamma=1/np.sqrt(1-beta_rel**2)
#beta_rel = np.sqrt(1 - 1/(gamma*gamma))
p0c = mass0*beta_rel*gamma #eV/c
Ek=(gamma-1)*mass0/1e6
E_tot=gamma*mass0
print('gamma',gamma)
print('beta',beta_rel)
print('Ek',Ek)
print('p0c',p0c/1e9)

current = 5*1e-3 #A
cooler_length = 2.33 # m 
radius_e_beam = 25*1e-3  #m

T_perp = 0.01 # <E> [eV] = kb*T
T_l =  0.001 # <E> [eV]
magnetic_field = 0.01  # T for ELENA
magnetic_field_ratio=0
q0=1

circumference = 30.40531277976 #m
T = circumference/(c*beta_rel)
s_per_turn = T

beta_x = 2.1 #m
beta_y = 2.2 #m
disp_x    = 1.5 #m
disp_x    = 0 #m


qx  = 2.36168984503
qy  = 1.38992572490
dqx = -25.93420025117
dqy = -14.90495652098

arc = xt.LineSegmentMap(
        qx=qx, qy=qy, qs=0,
        dqx=dqx, dqy=dqy,
        bets = 1*1e40, length=circumference,
        betx=beta_x,
        bety=beta_y
        )
num_particles=int(1e6)


emittance=10*1e-6 #inital emittance
px=np.random.normal(0, 4*np.sqrt(emittance/beta_x), num_particles)
dtk_particle = dtk.TestParticles(
    
    mass0=mass0,
    p0c=p0c,
    x=np.random.normal(0,1e-20 ,num_particles),
    px=px,
    y=np.random.normal(0, 1e-20 ,num_particles),
    py=np.random.normal(0, 1e-20 ,num_particles),
    delta=np.random.normal(0, 1e-20 ,num_particles),
    zeta=np.random.normal(0, 0 ,num_particles),
    q0=q0)

dtk_cooler = dtk.elements.ElectronCooler(current=current, length=cooler_length, radius_e_beam=radius_e_beam,
                            temp_perp=T_perp, temp_long=T_l,
                            magnetic_field=magnetic_field, magnetic_field_ratio=magnetic_field_ratio,
                            neutralisation_space_charge=0)
force,Fy,Fl = dtk_cooler.force(dtk_particle)

def temperature_to_velocity_1D(temperature_eV):
    
    T=temperature_eV#J
    mass = 511 *1e3
    velocity = c*np.sqrt( T/ mass)
    return velocity

px_tot = p0c*dtk_particle.px
beta_diff = px_tot/(mass0*gamma)
v_diff = beta_diff*c
# Sort v_diff and force in ascending order
sort_indices = np.argsort(v_diff)
v_diff = v_diff[sort_indices]
force = force[sort_indices]

filepath='results/force_xsuite.npz'
np.savez(filepath, v_diff=v_diff, force=force)


gamma 1.0006852036131262
beta 0.037
Ek 0.6429074249026815
p0c 0.034739854836641396
